In [1]:
#data preprocessing for feeding into lstm

import cv2
import os


dataset=[]


def load_images_from_folder(folder,x):
    
    image=[]
    im=[]
    c=1
    
    
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        n_img=cv2.resize(img,(100,100))
        if img is not None :
            im.append(n_img)
        if c%10==0:
            image.append(im)
            image.append(x)
            im=[]
            dataset.append(image)
            image=[]
        c=c+1
        
load_images_from_folder( "./HockeyFightFrames/training/nonViolence",0)
load_images_from_folder( "./HockeyFightFrames/training/violence",1)

In [2]:
print(len(dataset))
# print(dataset)

import random
train_data = []
test_data = []

3694


In [3]:
import random
random.shuffle(dataset)
for samples in dataset[:10]:
    print(samples[1])
x=[]
y=[]
for img,lab in dataset:
    x.append(img)
    y.append(lab)

0
0
1
1
0
0
1
0
1
1


In [4]:
import numpy as np
x=np.array(x).reshape(-1,10,100,100,3)
print(x.shape)

(3694, 10, 100, 100, 3)


In [5]:
#saving everything as pickle
import pickle
pickle_out=open("xhockeyGRU.pickle","wb")
pickle.dump(x,pickle_out)
pickle_out.close()


pickle_out=open("yhockeyGRU.pickle","wb")
pickle.dump(y,pickle_out)
pickle_out.close()

In [6]:
#loading from pickle
import pickle
pickle_in=open("xhockeyGRU.pickle","rb")
x=pickle.load(pickle_in)
pickle_in.close()

pickle_in=open("yhockeyGRU.pickle","rb")
y=pickle.load(pickle_in)
pickle_in.close()

In [7]:
#data preprocessing for feeding into lstm

import cv2
import os


validset=[]


def load_images_from_folder(folder,x):
    
    image=[]
    im=[]
    c=1
    
    
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        n_img=cv2.resize(img,(100,100))
        if img is not None :
            im.append(n_img)
        if c%10==0:
            image.append(im)
            image.append(x)
            im=[]
            validset.append(image)
            image=[]
        c=c+1
        
load_images_from_folder( "./HockeyFightFrames/testing/nonViolence",0)
load_images_from_folder( "./HockeyFightFrames/testing/violence",1)

In [8]:
print(len(validset))

410


In [9]:
import random
random.shuffle(validset)
for samples in validset[:10]:
    print(samples[1])
x_valid=[]
y_valid=[]
for img,lab in validset:
    x_valid.append(img)
    y_valid.append(lab)

1
1
0
0
0
1
0
1
0
0


In [10]:
import numpy as np
x_valid=np.array(x_valid).reshape(-1,10,100,100,3)
print(x_valid.shape)

(410, 10, 100, 100, 3)


In [11]:
x_valid=x_valid.astype('float32')/255
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard

In [12]:
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, TimeDistributed,Dropout, Activation, Flatten,Conv2D, MaxPooling2D,LSTM,Bidirectional
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.applications import MobileNetV2
from keras.layers import GRU

# K.image_data_format('tf')
import numpy as np
x=x.astype('float32')/255
NAME="HOCKEYFINALGRU"
tensorboard=TensorBoard(log_dir='logs/{}'.format(NAME))

mobilenet = MobileNetV2(
    input_shape=None,
    include_top= False,
    weights=None,
    input_tensor=None,
    pooling=None,
)

## training the CNN
cnn = Sequential()

cnn.add(mobilenet)

#converting to 1-d tensor
cnn.add(Flatten())

model=Sequential()
model.add(TimeDistributed(cnn,input_shape=x.shape[1:]))

model.add(GRU(32))

model.add(Dropout(0.25))

model.add(Dense(64,activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(32,activation='relu'))
model.add(Dropout(0.25))
    
model.add(Dense(2,activation='sigmoid'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='SGD',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 10, 11520)        2257984   
 ibuted)                                                         
                                                                 
 gru (GRU)                   (None, 32)                1109184   
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 64)                2112      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                      

In [ ]:
x = np.array(x)
y = np.array(y)
x_valid = np.array(x_valid)
y_valid = np.array(y_valid)

H = model.fit(x,y,epochs=30,validation_data=(x_valid,y_valid),batch_size=20, callbacks=[tensorboard])
model.save("HockeyGRU.h5")

Epoch 1/30
185/185 [==============================] - 1419s 8s/step - loss: 0.4736 - accuracy: 0.7702 - val_loss: 0.7003 - val_accuracy: 0.5000
Epoch 2/30
185/185 [==============================] - 1308s 7s/step - loss: 0.2407 - accuracy: 0.9182 - val_loss: 0.7630 - val_accuracy: 0.5000
Epoch 3/30
185/185 [==============================] - 1323s 7s/step - loss: 0.1748 - accuracy: 0.9450 - val_loss: 0.7500 - val_accuracy: 0.5000
Epoch 4/30
185/185 [==============================] - 1299s 7s/step - loss: 0.1296 - accuracy: 0.9597 - val_loss: 0.8484 - val_accuracy: 0.5000
Epoch 5/30
185/185 [==============================] - 1308s 7s/step - loss: 0.1034 - accuracy: 0.9659 - val_loss: 1.0609 - val_accuracy: 0.5000
Epoch 6/30
 59/185 [========>.....................] - ETA: 2:42:58 - loss: 0.1015 - accuracy: 0.9636